<a href="https://colab.research.google.com/github/davidnoone/ExampleData/blob/main/Simple_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NetCDF and plotting example.

This example shows how to read data from a netcdf file, and how to make some simple plots. 

**Learning goals:**
 * Utilize notebook framework
 * Read data stored in netcdf format
 * Produce a contour map

**Python modules**
* [numpy](https://numpy.org/doc/stable/user/) for manipulating arrays of gridded data
* [netCDF4](https://unidata.github.io/netcdf4-python/) module for reading NetCDF files
* [Matplotlib](https://matplotlib.org/3.1.1/index.html) library for displaying graphics
* (Bonus: [cartopy](https://scitools.org.uk/cartopy/docs/latest/) for managing fancy map projections)




**Using this notebook**: Remember to save your own copy! Once this note book has loaded, save a copy to your google grive or similar location. Otherwise, you will lose any code work you do when you reload. 

**Notebooks**: Notice that `netCDF4` and `cartopy` is not installed by default on `colab` so must be installed for each session.
Takes a few moments.


In [ ]:
# One could preinstall these (code below tries to be more intellegent)
#!pip install netCDF4
#!pip install cartopy
#!pip uninstall shapely -y
#!pip install shapely --no-binary shapely

In [ ]:
#
# Import the needed modules: Install those which are not present.
#

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

try:
  import netCDF4 as nc
except:
  !pip install netCDF4
  import netCDF4 as nc

try:
  import cartopy.crs as ccrs
except:
  !pip install cartopy
  !pip uninstall shapely -y
  !pip install shapely --no-binary shapely
  import cartopy.crs as ccrs


print("All modules needed have been imported. Ready to continue!")

## Obtaining the data

NetCDF is a standard data format which is used widely for atmospehric and oceanic data sets. The data format is structured, and remarkably convienient once youget te hang of it. 

Data sets have dimensions (number of latitude, number of longitudes) and variables (temperature). Some variables describe the coordinates (i.e., longitude and latitude values).


In [ ]:
# Download a data file from github, if it doesn't exist
!test ! -f ERA5_pr_2021-01-01.nc && wget 'https://raw.github.com/davidnoone/GEOPHYS_NOTEBOOKS/main/data/ERA5_pr_2021-01-01.nc'

In [ ]:
# Code example to read data from a netcdf file.
# Some knowledge of the file structure and vaiables names in the file
# are needed. (They are ways to get this from within python)
gravity = 9.81

# Set a filename: This is the file we just downloaded. 
filename = "ERA5_pr_2021-01-01.nc"

# Open the netcdf file
ds = nc.Dataset(filename,'r')

# Read the coodinate data
lons = ds.variables["longitude"][:]
lats = ds.variables["latitude"][:]
levs = ds.variables["level"][:]

nlon = len(lons)
nlat = len(lats)

# Read the temperature and wind data
k = 6         # choose which pressure level to read
print('Reading data at pressure:',levs[k])
temp = ds.variables["t"][0,k,:,:]
uwnd = ds.variables["u"][0,k,:,:]
vwnd = ds.variables["v"][0,k,:,:]
zgeo = ds.variables["z"][0,k,:,:]/gravity

# It is good manners to close the file
ds.close()

print("Finished reading netcdf file.")


# Check the data

Convince yourself that the latitudes and longitudes were read in correctly. Print our their values to the screeed.
(You could do this with temperature as well, but it wil be many many values).


In [ ]:
# Print the dimensions to the screen
print("Number of longitudes: nlon=", nlon)
print("Number of latitudes : nlat=", nlat)

# Print the latitude and longitude 
print('LONGITUES:',lons)
print('LATITUDES:',lats)

print('You selected level k=',k,' which is',levs[k])



##Construct a map of temperature

To convince yourself that the data is good, and get a view of the atmospheric conditions, construct a contour map of the data showing the temperature. We're using matplotlib to do the plotting. 


For geophysical data, we also need to set up a map projection. The easiest map projection is cylindrical equidistant: treating latitude and longitude as cartesian coordinates. This works find for many instances, but does not work well near the poles. 

As a bonus, you might try to overlay the wind data as vectors do visualize the airflow. Do do this you can use the quiver function.

Also see documentation for matplotlib functions:
* [contour](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html) 
* [contourf](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html)
*[quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html)
* [colorbar](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.colorbar.html)

A [good example](https://scitools.org.uk/cartopy/docs/v0.13/matplotlib/advanced_plotting.html) is on the cartopy documentation. You can limit the view with the "extent" keyword to give latitude and longitude bounds. Try limiting the map to just a local reagion.


In [ ]:
# Simpliest possible case: Filled contour plot.

p = plt.contourf(lons, lats, temp)


In [ ]:
## A more aestetically pleasing case: 
##   * Makes it bigger
##   * Uses map projection to give continental outlines/coastlines
##   * Adds a color bar
##   * Overlays vectors

## Or here, with a map projection
fig = plt.figure(figsize=[12,8])   # make it bigger

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title("Temperature and wind vectors")

# Shaded contours for temperature
p = plt.contourf(lons, lats, temp,60, transform=ccrs.PlateCarree())
## Add a color bar
plt.colorbar(p)


## Try this to get vectors showing the streamlines
#v = ax.quiver(lons, lats, uwnd, vwnd, transform=ccrs.PlateCarree())  # easiest version
#v = ax.streamplot(lons, lats, uwnd, vwnd, transform=ccrs.PlateCarree()) # try this instread of quiver

step = 6    # plots every "stepth" vector
v = ax.quiver(lons[::step], lats[::step], uwnd[::step,::step], vwnd[::step,::step], transform=ccrs.PlateCarree())


# Add coastlines
ax.coastlines()



In [ ]:
# Lets redo the above with a few changes:
#
#  Change the colormap: cmap="name_of_color_map"
#  Limit the extent of the plot: extent = [lat1,lat2,lon1,lon2]
#  Overlay multiple fields.
#
#  Notice also the ERA5 data spans from -180 to +180 longitude. NZ is on the date line
#  So we need to "wrap" the data around.
#

extent = [150, 200, -65, -25]
proj_grid = ccrs.PlateCarree(180)      # Map projection with 180 is the central longitude
proj_data = ccrs.PlateCarree(0)      # Data has "longitude", relative to 0 E.

fig = plt.figure(figsize=[12,8])   # make it bigger

ax = plt.axes(projection=proj_grid)
ax.set_extent(extent,proj_data)
p = plt.contourf(lons, lats, temp,60, transform=proj_data,cmap='jet')
ax.coastlines()

plt.colorbar(p)

fig.show()




In [ ]:

# Redo the above, but add a "cycling point" to the to avoid the gap
import cartopy.util as cutil
ctemp, clon, clat = cutil.add_cyclic(temp, lons, lats)
czgeo, clon, clat = cutil.add_cyclic(zgeo, lons, lats)

#extent = [0, 360, -90, +90]         # Global
#extent = [90, 270, -80, +50]         # Pacific
extent = [150, 200, -65, -25]       # NZ
proj_grid = ccrs.PlateCarree(180)      # Map projection with 180 is the central longitude
proj_data = ccrs.PlateCarree(0)        # Data has "longitude", relative to 0 E.

fig = plt.figure(figsize=(12,8))        # make it bigger
ax = plt.axes(projection=proj_grid)

ax.set_title('['+filename+'] Temperature [K] and geopotential height [m]   LEVEL ='+str(levs[k])+' hPa')
ax.set_extent(extent,proj_data)

# Temperature as color fill
pt = plt.contourf(clon, clat, ctemp,60, transform=proj_data,cmap='jet')

# geopotential height as black confours
pz = plt.contour (clon, clat, czgeo,32, transform=proj_data,colors='black')

# Wind vectors on top (note, wrapping not needed)
step = 2
scale = 10                            # make this bigger for smaller arrows
pv = ax.quiver(lons[::step], lats[::step], uwnd[::step,::step], vwnd[::step,::step],\
              scale_units="xy",scale=scale,transform=proj_data,color='blue')

ax.clabel(pz)
plt.colorbar(pt)
ax.coastlines()

fig.show()



##Outcome

Being able to obtain gridded datasets, visualize fields and manipulate the data is a fundemental activity in geophsyical fluids analysis. Having completed this notebook task, you've developed some essential building blocks on which more complex analysis can rely. 

All the of the modules used here have greater power than what we've looked at more details can be found at:

* [Numpy manual](https://numpy.org/doc/stable/user/)
* [NetCDF4 documentation](https://unidata.github.io/netcdf4-python/ )
* [Matplotlib](https://matplotlib.org/3.1.1/index.html)
* [Cartopy manual](https://scitools.org.uk/cartopy/docs/latest/)

There are many examples on these pages, which is an excellent resource for more complicated coding.




In [ ]:
# Clean up data file
#!rm ERA5_pr_2021-01-01.nc
print('Finished')